In [1]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.optimize import curve_fit as fit

In [2]:
t = -0.5
delta = 1.0
alpha = [0.5,1.0]
L = [14]
mode = 's'
taus_all = np.logspace(-1,4,300)

max_tau_idx = np.sum(np.where(taus_all<5,1,0))
taus = taus_all[1:max_tau_idx]
inv_L = [1/i for i in L]

In [3]:
def import_eigval(full_path) -> np.ndarray:
    with open(full_path, 'r') as f:
        # print("Opened")
        flag = False
        data = []
        for line in f:
            if line.startswith('#Correlation matrix eigenvalues'):
                flag = True
                continue
            if flag == False:
                continue
            if line.startswith('#Eigenvector') or line.startswith('#5 eigenvectors'):
                return np.array(data, dtype=np.float64)
            try:
                data.append(float(line))
            except:
                continue
    return np.ones(1)*-1

def fun(x,gamma): 
    # x = 1/(tau*alpha^2)
    return 2/np.pi * np.arctan(x/gamma)

In [4]:

eigval_int = np.zeros(len(L))
for i,l in enumerate(L):
    int_path = 'L_' + str(l) + '_m_3_t_' + '{:.1f}'.format(t) + '_delta_' + '{:.1f}'.format(delta) + '_alpha_0.00.dat'
    if mode == 'ec':
        int_path = 'data/preprocessed/' + 'energy_current_' + int_path
    elif mode == 's':
        int_path = 'data/preprocessed/' + 'spin_' + int_path
    eigval_int[i] = import_eigval(os.path.abspath(int_path))[-1]
    
if len(L) > 1:
    fit_int = np.polyfit(inv_L,eigval_int,1)
    eigval_int = fit_int[-1]
    if eigval_int > 1.0: eigval_int = 1.0
    if eigval_int < 0.0: eigval_int = 0.0
    print("Extrapolated integrable lambda = ",eigval_int)
else:
    eigval_int = eigval_int[0]
    print('integrable lambda for L = '+str(L[0])+': ',eigval_int)
    

integrable lambda for L = 14:  1.00000000000002


In [5]:

data_fs = np.empty((len(L),len(alpha)),dtype=np.ndarray)
data_ft = np.empty((len(L),len(alpha)),dtype=np.ndarray)
for i,l in enumerate(L):
    for j,a in enumerate(alpha):
        fixed_spacing_name = 'data/preprocessed/' + mode + '_fixed_mat_el_spacing_L_' + str(l) + '_d_' + '{:.1f}'.format(delta) + \
        '_a_' + '{:.2f}'.format(a) + '.csv'
        fixed_tau_name = 'data/preprocessed/' + mode + '_fixed_tau_L_' + str(l) + '_d_' + '{:.1f}'.format(delta) + \
        '_a_' + '{:.2f}'.format(a) + '.csv'
        data_fs[i,j] = pd.read_csv(os.path.abspath(fixed_spacing_name)).to_numpy()
        data_ft[i,j] = pd.read_csv(os.path.abspath(fixed_tau_name)).to_numpy()

In [6]:

eigval_noint = np.zeros((len(alpha),len(taus)))
if(len(L) > 1):
    for k in range(len(alpha)):
        eigval_L_tau = np.zeros((len(L),len(taus)))
        for i,l in enumerate(L):
            for j,tau in enumerate(taus):
                eigval_L_tau[i,j] = data_ft[i,k][j,1]
            
        for j in range(len(taus)):
            fit_noint = np.polyfit(inv_L,eigval_L_tau[:,j],1)
            eigval_noint[k,j] = fit_noint[-1]
            # print(eigval_noint[k,j])
            if eigval_noint[k,j] > 1.0 : eigval_noint[k,j] = 1.0
            if eigval_noint[k,j] < 0.0 : eigval_noint[k,j] = 0.0
else:
    for k in range(len(alpha)):
        for j,tau in enumerate(taus):
            eigval_noint[k,j] = data_ft[0,k][j,1]
            # print(eigval_noint[k,j])
            

In [7]:
R1 = np.zeros((len(alpha),len(taus)))
for i in range(len(alpha)):
    for j in range(len(taus)):
        R1[i,j] = eigval_noint[i,j]/eigval_int

In [8]:
x = np.zeros((len(alpha),len(taus)))
scaling_factor = 1
for i,a in enumerate(alpha):
    x[i,:] = 1/(taus*(a**scaling_factor))

for i,a in enumerate(alpha):        
    popt = fit(fun,x[i,:],R1[i,:])[0]
    label = r'$\alpha = ' + r'{:.2f}'.format(a) + r'$, ' + r'$\gamma = ' +r'{:.3f}'.format(popt[0])+ '$'
    p = plt.plot(x[i,:],R1[i,:],'x:',markersize=5,label=label)
    xx = np.linspace(x[i,0],x[i,-1],100)
    y = fun(xx,*popt)
    plt.plot(xx,y,'-',color=p[-1].get_color(),linewidth=1.0)
plt.xlabel(r'$\frac{1}{\tau \alpha^2}$',fontsize=16)
plt.ylabel(r'$R_1$',fontsize=16)
plt.legend(loc='best',fontsize=12)
plt.ylim([0.0,1.05])
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …